1. make sure that the random seeds are working -
2. run the training for "5" epochs and for each set of hyper-parameters run if e.g. 10 times (each time with different random seed) -
3. learn how to plot the "evolution", i.e. training/testing error vs. epochs for mutliple runs  (use https://seaborn.pydata.org/) -
4. multiple lines will corespond to different "hyper-parameters" ("lr" and maybe "momentum" (0, 0.9, 0.99) ) -
5. try to also compare with SignSGD with two versions of SignSGD with momentum -

In [0]:
! nvidia-smi | grep -A 2 -B 2 "N/A"

|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:04:00.0 Off |                    0 |
| N/A   43C    P0    57W / 149W |  11238MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00000000:05:00.0 Off |                    0 |
| N/A   36C    P0    74W / 149W |   1901MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  Tesla K80           Off  | 00000000:83:00.0 Off |                    0 |
| N/A   41C    P0    58W / 149W |   9655MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   3  Tesla K80           Off  | 00000000:84:00.0 Off |                    0 |
| N/A   26C    P8    30W / 149W |     11MiB / 11439MiB |      0%      Default |
+-------------------------------+-------

In [0]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="2"  # which GPU are we using (from 0 to 3)
import torch
torch.set_num_threads(0)
device="cuda:2"

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import numpy as np
import copy
import pickle
from pandas.core.frame import DataFrame
from torch.autograd import Variable
torch.cuda.is_available()

True

In [0]:
#! rm logs/*

In [0]:
#! ls logs/

In [0]:
#device="cpu"    
def computeErrorForWholeDataset(dataset):
    iteration=0
    with torch.no_grad():
        losses=[]
        for i, (images,labels) in enumerate(dataset):
            iteration+=1

            images =Variable(images).to(device)
            labels =Variable(labels).to(device)

            outputs=model(images)
            loss=loss_function(outputs,labels)
            losses.append(loss.item())
    return np.mean(losses)

In [0]:
from classes.models.CNN import CNN
from classes.optim import SGD
from classes.optim import signSGDm
L=[[2**-i,0] for i in range(1,11)] #set a list of diminishing learing rates
epochs=5

In [0]:
for eta, momentum  in L:
    for experiment in range(10):
        
        logfile = "logs/signSGDm_seed_%d_lr_%f_momentum_%f.log"%(experiment, eta, momentum)
        print logfile
        if os.path.exists(logfile):
            print  "skipping..."
            continue
        
        torch.manual_seed(experiment)
        torch.cuda.manual_seed(experiment)

        train_dataset= datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
        test_dataset= datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

        train_load=torch.utils.data.DataLoader(dataset=train_dataset, batch_size=1000, shuffle=True)
        test_load=torch.utils.data.DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)

        #train_loadLB=torch.utils.data.DataLoader(dataset=train_dataset, batch_size=1000, shuffle=True)
        #test_loadLB=torch.utils.data.DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)

        model=CNN()
        CUDA=torch.cuda.is_available()
        if CUDA:
            model=model.cuda()
        loss_function=nn.CrossEntropyLoss()
        #optimizer=SGD.SGD(model.parameters(), lr=eta,momentum=momentum)
        optimizer=signSGDm.signSGDm(model.parameters(),lr=eta,momentum=momentum)
        # RUN IT
        loss_list=[]
        for epoch in range(epochs):
            for i, (images,labels) in enumerate(train_load):  
                if CUDA:
                    images =Variable(images.cuda())
                    labels =Variable(labels.cuda())
                else:
                    images =Variable(images)
                    labels =Variable(labels)

                if i% (len(train_load)/5) == 0:
                    train_loss=computeErrorForWholeDataset(train_load)
                    test_loss=computeErrorForWholeDataset(test_load)  # compute errors train and test .....
                    #loss_list.append([train_loss,test_loss,epoch,'SGD',experiment,eta,momentum])
                    loss_list.append([train_loss,test_loss,epoch,'signSGDm',experiment,eta,momentum])
                optimizer.zero_grad()
                outputs=model(images)
                loss=loss_function(outputs,labels)
                loss.backward()
                optimizer.step()
        pickle.dump(loss_list,open(logfile,'wb'))
print("Finished!")

logs/signSGDm_seed_0_lr_0.500000_momentum_0.000000.log
logs/signSGDm_seed_1_lr_0.500000_momentum_0.000000.log
logs/signSGDm_seed_2_lr_0.500000_momentum_0.000000.log
logs/signSGDm_seed_3_lr_0.500000_momentum_0.000000.log
logs/signSGDm_seed_4_lr_0.500000_momentum_0.000000.log
logs/signSGDm_seed_5_lr_0.500000_momentum_0.000000.log
logs/signSGDm_seed_6_lr_0.500000_momentum_0.000000.log
logs/signSGDm_seed_7_lr_0.500000_momentum_0.000000.log
logs/signSGDm_seed_8_lr_0.500000_momentum_0.000000.log
logs/signSGDm_seed_9_lr_0.500000_momentum_0.000000.log
logs/signSGDm_seed_0_lr_0.250000_momentum_0.000000.log
logs/signSGDm_seed_1_lr_0.250000_momentum_0.000000.log
logs/signSGDm_seed_2_lr_0.250000_momentum_0.000000.log
logs/signSGDm_seed_3_lr_0.250000_momentum_0.000000.log
logs/signSGDm_seed_4_lr_0.250000_momentum_0.000000.log
logs/signSGDm_seed_5_lr_0.250000_momentum_0.000000.log
logs/signSGDm_seed_6_lr_0.250000_momentum_0.000000.log
logs/signSGDm_seed_7_lr_0.250000_momentum_0.000000.log
logs/signS

The following will kill the kernel and will free memory on GPU for others to use

In [0]:
exit(0)